In [1]:
from scapy.all import *
import ipaddress
import random
import os
import sys
import binascii
import socket
import hashlib

if os.geteuid() != 0:
    raise RuntimeError('Run it as root')
if sys. version_info.major != 3:
    raise RuntimeError('Run it with python 3')

# Preparation
## PHI header

In [2]:
def make_phi_packet(phi_header, pktgen_header=False, mtu=0):
    payload = bytes()    
    
    if phi_header['mode'] != 3:
        phi_header['handshake_phase'] = 0
    if not 'forward_stack' in phi_header:
        phi_header['forward_stack'] = [struct.pack('B' * 8, *[random.randint(0, 255) for i in range(8)]) for j in range(16)]
    if not 'plain_dst' in phi_header:
        phi_header['plain_dst'] = '0.0.0.0'
    
    # mode, phase
    payload += struct.pack('B', (phi_header['mode'] << 6) + (phi_header['handshake_phase'] << 4))
    payload += binascii.unhexlify('FF' * 11)
    # plain_dst
    payload += binascii.unhexlify('FF' * 4)
    payload += socket.inet_aton(phi_header['plain_dst'])
    # forward_stack
    for item in phi_header['forward_stack']:
        item_bytes = binascii.unhexlify(item)
        payload += item_bytes
    # encrypted_dst
            
    if mtu != 0:
        payload += binascii.unhexlify('00' * (mtu - len(payload)))            
            
    if pktgen_header:
        pktgen_hdr = binascii.unhexlify('010000000000')
        pkt = Raw(load=pktgen_hdr)/Ether()/Raw(load=payload)
    else:
        pkt = Ether()/Raw(load=payload)
        
    pkt[Ether].dst = "FF:FF:FF:FF:FF:FF"
    pkt[Ether].type = 0xFFFF
    return pkt

def parse_phi_packet(pkt):
    phi_header = {}
    payload = bytes(pkt[Raw])

    # mode, phase
    idx = 0
    phi_header['mode'] = payload[idx] >> 6
    phi_header['handshake_phase'] = payload[idx] % (1 << 6) >> 4
    idx += 1
    idx += 11
    idx += 4
    # plain_dst
    phi_header['plain_dst'] = str(ipaddress.ip_address(payload[idx:idx + 4]))
    idx += 4
    # forward_stack
    phi_header['forward_stack'] = []
    for i in range(16):
        phi_header['forward_stack'].append(binascii.hexlify(payload[idx:idx + 8]))
        idx += 8
    # encrypted_dst  
    phi_header['encrypted_dst'] = payload[idx:idx + 128]
    idx += 128
    
    return phi_header

def print_phi_packet(phi_header):
    mode_str = ['forward data', 'backward data', 'VALUE_ERROR', 'handshake']
    handshake_phase_str = ['source -> helper', 'helper -> midway', 'midway -> destination', 'destination -> source']

    print('mode:', phi_header['mode'], ' (%s)' % mode_str[phi_header['mode']])
    if phi_header['mode'] == 3:
        print('handshake_phase:', phi_header['handshake_phase'], ' (%s)' % handshake_phase_str[phi_header['handshake_phase']])
        
    if phi_header['mode'] == 3 and phi_header['handshake_phase'] != 3:
        print('plain_dst:', phi_header['plain_dst'])

    if phi_header['mode'] == 3 and phi_header['handshake_phase'] == 0:
        print('encrypted_dst:', 'ENCRYPTED')        

    print('forward_stack:')
    for i in range(16):
        print(phi_header['forward_stack'][i])

## HalfSipHash

In [3]:
CONST_0 = 0x70736575
CONST_1 = 0x6e646f6d
CONST_2 = 0x6e657261
CONST_3 = 0x79746573

def rol(x, n):
    y = (x & (2**n) - 1 << (32 - n)) >> (32 - n)
    z = (x & (2**(32-n) - 1)) << n
    return y | z

def sip_round():
    global v0_0, v0_1, v0_2, v0_3, v1_0, v1_1, v1_2, v1_3
    
    v1_0 = (v0_0 + v0_1) % (1 << 32)
    v1_2 = (v0_2 + v0_3) % (1 << 32)
    v1_1 = rol(v0_1, 5)
    v1_3 = rol(v0_3, 8)
    
    v0_1 = v1_1 ^ v1_0
    v0_3 = v1_3 ^ v1_2
    v0_0 = rol(v1_0, 16)
    v0_2 = v1_2
    
    v1_2 = (v0_2 + v0_1) % (1 << 32)
    v1_0 = (v0_0 + v0_3) % (1 << 32)
    v1_1 = rol(v0_1, 13)
    v1_3 = rol(v0_3, 7)
    
    v0_1 = v1_1 ^ v1_2
    v0_3 = v1_3 ^ v1_0
    v0_2 = rol(v1_2, 16)
    v0_0 = v1_0

def sip_hash(key0, key1, message):
    global v0_0, v0_1, v0_2, v0_3, v1_0, v1_1, v1_2, v1_3
    
    v0_0 = CONST_0 ^ key0
    v0_1 = CONST_1 ^ key1
    v0_2 = CONST_2 ^ key0
    v0_3 = message ^ key1
    
    sip_round()
    sip_round()
    
    v0_0 = v0_0 ^ message ^ CONST_3
    v0_2 = v0_2 ^ 0xFF
    
    sip_round()
    sip_round()
    sip_round()
    sip_round()
    
    return v0_0 ^ v0_1 ^ v0_2 ^ v0_3    

## FIB partitioning

In [4]:
fib = {}
with open('bgp_fib/rib_wain.txt') as f:
    for line in f:
        line = line.rstrip()
        
        prefix = ipaddress.ip_network(line.split(' ')[0], strict=False)
        netaddr = prefix.network_address
        netmask = prefix.netmask
        fib[(netaddr, netmask)] = 0
        

out_ports = list([i for i in range(0, 64, 8)] + [i for i in range(128, 192, 8)] + \
    [i for i in range(256, 320, 8)] + [i for i in range(384, 448, 8)])
for i, key in enumerate(fib):
    h = hashlib.md5((str(key[0]) + str(key[1])).encode('utf-8')).hexdigest()
    fib[key] = out_ports[int(h, 16) % len(out_ports)]

## Trie

In [5]:
# 高速にLPMマッチをするためにTrie木を実装する
class IPLPM:
    def __init__(self):
        self.root = {0: None}

    def __insert_rec(self, node, ip_int, prefix_len, value):
        if prefix_len == 0:
            node[0] = value
            while len(node) < 3:
                node[len(node)] = {0: value}
            return
        while len(node) < 3:
            node[len(node)] = {0: None}
        b = 1 if ip_int >= (1 << 31) else 0
        ip_int = ip_int * 2 % (1 << 32)
        self.__insert_rec(node[1 + b], ip_int, prefix_len - 1, value)

    def __setitem__(self, key, value):
        (ip_prefix, prefix_len) = key
        ip_int = int(ipaddress.ip_address(ip_prefix))
        self.__insert_rec(self.root, ip_int, prefix_len, value)

    def __get_rec(self, node, ip_int, prefix_len):
        if len(node) == 1:
            return node[0]
        elif prefix_len == 0:
            return None
        b = 1 if ip_int >= (1 << 31) else 0
        ip_int = ip_int * 2 % (1 << 32)
        return self.__get_rec(node[1 + b], ip_int, prefix_len - 1)
        
    def __getitem__(self, ip_addr):
        ip_int = int(ipaddress.ip_address(ip_addr))
        res = self.__get_rec(self.root, ip_int, 32)
        return res

prefix_list = list(fib.keys())
def get_rand_ip():
    prefix = random.choice(prefix_list)
    netaddr = int(prefix[0])
    netmask = int(prefix[1])

    subnet = random.randint(0, (1 << 32) - 1)
    subnet = subnet & ~netmask
    rand_ip = ipaddress.IPv4Address(netaddr | subnet)
    
    return rand_ip

In [6]:
lpm = IPLPM()
for key in fib:
    lpm[str(key[0]), bin(int(key[1])).count('1')] = fib[key]

# PHI demo

![](./figure/phi_demo.png)

In [7]:
key0 = [0x01234567] * 16
key1 = [0x12345678] * 16

dest_addr = str(get_rand_ip())
while lpm[dest_addr] == 0:
    dest_addr = str(get_rand_ip())
    
helper_addr = str(get_rand_ip())
while lpm[helper_addr] == 0:
    helper_addr = str(get_rand_ip())
    
print('destination:', dest_addr)
print('helper:', helper_addr)

destination: 141.207.200.213
helper: 65.22.69.239


## Handshake phase 1

In [8]:
phi_header = {}
phi_header['mode'] = 3
phi_header['handshake_phase'] = 0
phi_header['plain_dst'] = helper_addr
phi_header['encrypted_dst'] = dest_addr

phi_header['forward_stack'] = ['FFFFFFFFFFFFFFFF'] * 16
# phi_header['forward_stack'] = [('%x' % random.randint(0, (1 << 64) - 1)).zfill(16) for i in range(16)]

pkt = make_phi_packet(phi_header)

expected_port = lpm[helper_addr]
print('expected port:', expected_port)

router_count = 3
for i in range(1, router_count + 1):
    print()
    print('----- Router %d -----' % i)
    
    sniffer = scapy.sendrecv.AsyncSniffer(count=4, iface='veth%d' % (out_ports.index(expected_port) * 2))
    sniffer.start()
    scapy.sendrecv.sendp(pkt, iface='veth0')
    time.sleep(2)
    pkts = sniffer.stop()
    
    print()
    sniffed = False
    for sniffed_pkt in pkts:
        sniffed_phi_header = parse_phi_packet(sniffed_pkt)
        if sniffed_phi_header['forward_stack'] != phi_header['forward_stack']:
            sniffed = True
            print('sniffed_on:', sniffed_pkt.sniffed_on)
            pkt = sniffed_pkt
            phi_header = sniffed_phi_header
            print_phi_packet(phi_header)
            break
    if not sniffed:
        raise Exception

expected port: 144

----- Router 1 -----

Sent 1 packets.

sniffed_on: veth20
mode: 3  (handshake)
handshake_phase: 0  (source -> helper)
plain_dst: 65.22.69.239
encrypted_dst: ENCRYPTED
forward_stack:
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'

----- Router 2 -----

Sent 1 packets.

sniffed_on: veth20
mode: 3  (handshake)
handshake_phase: 0  (source -> helper)
plain_dst: 65.22.69.239
encrypted_dst: ENCRYPTED
forward_stack:
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'fffffffffffff

In [9]:
stack_depth = 3
for i in range(stack_depth):
    stack_item = binascii.unhexlify(sniffed_phi_header['forward_stack'][i])
    ports = int.from_bytes(stack_item[0:4], 'big')
    nonce = int.from_bytes(stack_item[4:8], 'big')
    otp = sip_hash(key0[i], key1[i], nonce)
    ingress = ((ports ^ otp) & 0x01FF0000) >> 16
    egress = (ports ^ otp) & 0x00001FF
    print('%d: %d -> %d' % (i, ingress, egress))

0: 0 -> 144
1: 0 -> 144
2: 0 -> 144


## Handshake phase 2

In [10]:
sniffed_phi_header['handshake_phase'] = 1
sniffed_phi_header['plain_dst'] = dest_addr
pkt = make_phi_packet(sniffed_phi_header)

expected_port = lpm[dest_addr]
print('expected port:', expected_port)

router_count = 3
for i in range(1, router_count + 1):
    print()
    print('----- Router %d -----' % i)
    
    sniffer = scapy.sendrecv.AsyncSniffer(count=4, iface='veth%d' % (out_ports.index(expected_port) * 2))
    sniffer.start()
    scapy.sendrecv.sendp(pkt, iface='veth0')
    time.sleep(2)
    pkts = sniffer.stop()
    
    print()
    sniffed = False
    for sniffed_pkt in pkts:
        sniffed_phi_header = parse_phi_packet(sniffed_pkt)
        if sniffed_phi_header['forward_stack'] != phi_header['forward_stack']:
            sniffed = True
            print('sniffed_on:', sniffed_pkt.sniffed_on)
            pkt = sniffed_pkt
            phi_header = sniffed_phi_header
            print_phi_packet(phi_header)
            break
    if not sniffed:
        raise Exception

expected port: 128

----- Router 1 -----

Sent 1 packets.

sniffed_on: veth16
mode: 3  (handshake)
handshake_phase: 2  (midway -> destination)
plain_dst: 141.207.200.213
forward_stack:
b'b924eb21a84d61df'
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'

----- Router 2 -----

Sent 1 packets.

sniffed_on: veth16
mode: 3  (handshake)
handshake_phase: 2  (midway -> destination)
plain_dst: 141.207.200.213
forward_stack:
b'7379ee23f87e917d'
b'b924eb21a84d61df'
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'

----- Ro

In [11]:
stack_depth = 5
for i in range(stack_depth):
    stack_item = binascii.unhexlify(sniffed_phi_header['forward_stack'][i])
    ports = int.from_bytes(stack_item[0:4], 'big')
    nonce = int.from_bytes(stack_item[4:8], 'big')
    otp = sip_hash(key0[i], key1[i], nonce)
    ingress = ((ports ^ otp) & 0x01FF0000) >> 16
    egress = (ports ^ otp) & 0x00001FF
    print('%d: %d -> %d' % (i, ingress, egress))

0: 0 -> 128
1: 0 -> 128
2: 0 -> 128
3: 0 -> 144
4: 0 -> 144


## Handshake phase 3

In [12]:
sniffed_phi_header['handshake_phase'] = 3
pkt = make_phi_packet(sniffed_phi_header)

expected_port = 0

router_count = 5
for i in range(1, router_count + 1):
    print()
    print('----- Router %d -----' % i)
    
    sniffer = scapy.sendrecv.AsyncSniffer(count=4, iface='veth%d' % (out_ports.index(expected_port) * 2))
    sniffer.start()
    scapy.sendrecv.sendp(pkt, iface='veth0')
    time.sleep(2)
    pkts = sniffer.stop()
    
    print()
    sniffed = False
    for sniffed_pkt in pkts:
        sniffed_phi_header = parse_phi_packet(sniffed_pkt)
        if sniffed_phi_header['forward_stack'] != phi_header['forward_stack']:
            sniffed = True
            print('sniffed_on:', sniffed_pkt.sniffed_on)
            pkt = sniffed_pkt
            phi_header = sniffed_phi_header
            print_phi_packet(phi_header)
            break
    if not sniffed:
        raise Exception


----- Router 1 -----

Sent 1 packets.

sniffed_on: veth0
mode: 3  (handshake)
handshake_phase: 3  (destination -> source)
forward_stack:
b'7379ee23f87e917d'
b'b924eb21a84d61df'
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'6ca542e572c32fd8'

----- Router 2 -----

Sent 1 packets.

sniffed_on: veth0
mode: 3  (handshake)
handshake_phase: 3  (destination -> source)
forward_stack:
b'b924eb21a84d61df'
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'6ca542e572c32fd8'
b'7379ee23f87e917d'

----- Router 3 -----

Sent 1 packets.

sniffed_on: veth0
mode: 3  (handshake)
hands

## Data transmission phase 1

In [13]:
sniffed_phi_header['mode'] = 0
sniffed_phi_header['handshake_phase'] = 0
pkt = make_phi_packet(sniffed_phi_header)

router_count = 5
for i in range(1, router_count + 1):
    print()
    print('----- Router %d -----' % i)
    
    stack_item = binascii.unhexlify(sniffed_phi_header['forward_stack'][-1])
    ports = int.from_bytes(stack_item[0:4], 'big')
    nonce = int.from_bytes(stack_item[4:8], 'big')
    otp = sip_hash(key0[-1], key1[-1], nonce)
    expected_port = (ports ^ otp) & 0x00001FF
    
    sniffer = scapy.sendrecv.AsyncSniffer(count=4, iface='veth%d' % (out_ports.index(expected_port) * 2))
    sniffer.start()
    scapy.sendrecv.sendp(pkt, iface='veth0')
    time.sleep(2)
    pkts = sniffer.stop()
    
    print()
    sniffed = False
    for sniffed_pkt in pkts:
        sniffed_phi_header = parse_phi_packet(sniffed_pkt)
        if sniffed_phi_header['forward_stack'] != phi_header['forward_stack']:
            sniffed = True
            print('sniffed_on:', sniffed_pkt.sniffed_on)
            pkt = sniffed_pkt
            phi_header = sniffed_phi_header
            print_phi_packet(phi_header)
            break
    if not sniffed:
        raise Exception


----- Router 1 -----

Sent 1 packets.

sniffed_on: veth20
mode: 0  (forward data)
forward_stack:
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'6ca542e572c32fd8'
b'7379ee23f87e917d'
b'b924eb21a84d61df'
b'c2144ef78fa3793d'

----- Router 2 -----

Sent 1 packets.

sniffed_on: veth20
mode: 0  (forward data)
forward_stack:
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'6ca542e572c32fd8'
b'7379ee23f87e917d'
b'b924eb21a84d61df'

----- Router 3 -----

Sent 1 packets.

sniffed_on: veth16
mode: 0  (forward data)
forward_stack:
b'b924eb21a84d61df'
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffff

## Data transmission phase 2

In [14]:
sniffed_phi_header['mode'] = 1
pkt = make_phi_packet(sniffed_phi_header)

expected_port = 0

router_count = 5
for i in range(1, router_count + 1):
    print()
    print('----- Router %d -----' % i)
    
    sniffer = scapy.sendrecv.AsyncSniffer(count=4, iface='veth%d' % (out_ports.index(expected_port) * 2))
    sniffer.start()
    scapy.sendrecv.sendp(pkt, iface='veth0')
    time.sleep(2)
    pkts = sniffer.stop()
    
    print()
    sniffed = False
    for sniffed_pkt in pkts:
        sniffed_phi_header = parse_phi_packet(sniffed_pkt)
        if sniffed_phi_header['forward_stack'] != phi_header['forward_stack']:
            sniffed = True
            print('sniffed_on:', sniffed_pkt.sniffed_on)
            pkt = sniffed_pkt
            phi_header = sniffed_phi_header
            print_phi_packet(phi_header)
            break
    if not sniffed:
        raise Exception


----- Router 1 -----

Sent 1 packets.

sniffed_on: veth0
mode: 1  (backward data)
forward_stack:
b'7379ee23f87e917d'
b'b924eb21a84d61df'
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'6ca542e572c32fd8'

----- Router 2 -----

Sent 1 packets.

sniffed_on: veth0
mode: 1  (backward data)
forward_stack:
b'b924eb21a84d61df'
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'ffffffffffffffff'
b'6ca542e572c32fd8'
b'7379ee23f87e917d'

----- Router 3 -----

Sent 1 packets.

sniffed_on: veth0
mode: 1  (backward data)
forward_stack:
b'c2144ef78fa3793d'
b'd566f125ff43f703'
b'ffffffffffffffff'
b'ffff

# IP demo

In [15]:
payload = bytes() 
payload += binascii.unhexlify('FF' * 256)
pkt = Ether()/IP(dst=dest_addr)/TCP()/payload

expected_port = lpm[dest_addr]
print('expected port:', expected_port)

sniffer = scapy.sendrecv.AsyncSniffer(count=4, iface='veth%d' % (out_ports.index(expected_port) * 2))
sniffer.start()
scapy.sendrecv.sendp(pkt, iface='veth0')
time.sleep(2)
pkts = sniffer.stop()

print()
sniffed = False
for sniffed_pkt in pkts:
    sniffed = True
    pkt.show()
    break
if not sniffed:
    raise Exception

expected port: 128

Sent 1 packets.

###[ Ethernet ]### 
  dst       = 00:a0:c9:8d:01:01
  src       = 00:28:f8:2f:f8:8c
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = None
     src       = 10.11.121.184
     dst       = 141.207.200.213
     \options   \
###[ TCP ]### 
        sport     = ftp_data
        dport     = http
        seq       = 0
        ack       = 0
        dataofs   = None
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = None
        urgptr    = 0
        options   = []
###[ Raw ]### 
           load      = '\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\